# 🧠 IA + Oracle: Hola mundo con Gemini API Key

Este notebook muestra cómo conectar una base de datos Oracle y generar consultas SQL desde lenguaje natural usando el modelo Gemini de Google vía su librería oficial `google.generativeai`.

🔑 Solo necesitas una clave API (`GOOGLE_API_KEY`).  
❌ No se requiere `project_id`, `region` ni cuenta de servicio.

Ideal para prototipos rápidos y prácticas educativas.

## 🔍 Comparativa: Vertex AI vs Gemini API Key

| Característica              | Vertex AI (AISuite)         | Gemini API Key (`google.generativeai`) |
|----------------------------|-----------------------------|----------------------------------------|
| Requiere `project_id`      | ✅ Sí                        | ❌ No                                   |
| Requiere `region`          | ✅ Sí                        | ❌ No                                   |
| Requiere cuenta de servicio| ✅ Sí                        | ❌ No                                   |
| Ideal para producción      | ✅                           | 🔸 Solo para prototipos y pruebas       |
| Simplicidad de uso         | 🔸 Más compleja              | ✅ Muy sencilla                         |

In [ ]:
# 🔐 Configurar Gemini con API Key
from dotenv import load_dotenv
import os
import google.generativeai as genai

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash")

In [ ]:
# 🔗 Conexión a Oracle
import oracledb

ORACLE_HOST = os.getenv("ORACLE_HOST")
ORACLE_PORT = os.getenv("ORACLE_PORT")
ORACLE_SID = os.getenv("ORACLE_SID")
ORACLE_USER = os.getenv("ORACLE_USER")
ORACLE_PASSWORD = os.getenv("ORACLE_PASSWORD")

dsn = oracledb.makedsn(ORACLE_HOST, ORACLE_PORT, sid=ORACLE_SID)
connection = oracledb.connect(user=ORACLE_USER, password=ORACLE_PASSWORD, dsn=dsn)
cursor = connection.cursor()

cursor.execute("SELECT USER FROM dual")
print("Conectado como:", cursor.fetchone()[0])

In [ ]:
# 🗂️ Descubrimiento del esquema
cursor.execute(f"""
    SELECT table_name, column_name
    FROM all_tab_columns
    WHERE owner = UPPER('{ORACLE_USER}')
    ORDER BY table_name, column_id
""")

esquema = {}
for table, column in cursor:
    esquema.setdefault(table, []).append(column)

print("Esquema de la base de datos:")
for tabla, columnas in esquema.items():
    print(f"{tabla}({', '.join(columnas)})")

In [ ]:
# 🧠 Pregunta en lenguaje natural
pregunta = "¿Qué productos compró Ana Gamez y cuánto costaron?"

In [ ]:
# 🤖 Generación de SQL con Gemini
esquema_texto = "\n".join(
    f"{tabla}({', '.join(columnas)})" for tabla, columnas in esquema.items()
)

prompt_sql = f"""
Eres un asistente experto en SQL para Oracle. Genera solo la consulta SQL compatible con Oracle.
Usa este esquema de base de datos:
{esquema_texto}

Pregunta del usuario:
{pregunta}
"""

raw_sql = model.generate_content(prompt_sql)
sql_generado = raw_sql.text.strip().strip("```sql").strip("```")
sql_generado = sql_generado.replace(";", "")  # Elimina el punto y coma si está presente
#eliminar saltos de línea y tabulaciones pues dan error en Oracle
sql_generado = sql_generado.replace("\n", " ").replace("\t", " ")


print("Consulta generada por IA:")
print(sql_generado)

In [ ]:
# 🧪 Ejecutar SQL y mostrar resultados
cursor.execute(sql_generado)
resultados = cursor.fetchall()
columnas = [col[0] for col in cursor.description]

import pandas as pd
df = pd.DataFrame(resultados, columns=columnas)
texto_resultado = df.to_markdown(index=False)
df.head()

In [ ]:
# 🧠 Interpretación con Gemini
prompt_explicacion = f"""
Eres un experto en análisis de datos. Resume e interpreta los resultados de una consulta SQL.
La pregunta original fue: {pregunta}
Los resultados fueron:

{texto_resultado}
"""

response = model.generate_content(prompt_explicacion)
print("🧠 Respuesta interpretada por IA:")
print(response.text.strip())

## 🧠 Reflexión

- ¿Qué ventajas tiene usar Gemini con API Key?
- ¿Qué limitaciones presenta frente a Vertex AI?
- ¿Cómo podrías extender este flujo para validar o visualizar los resultados?

Este notebook es ideal para prácticas rápidas, pero recuerda que para entornos profesionales, Vertex AI ofrece mayor control y seguridad.